In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
from IPython.display import display

# %matplotlib inline

from pathlib import Path


In [2]:
def collect_one(run, csv_folder):
    with open(run / 'config.json', 'r') as f:
        cfg = json.load(f)
        
    run_dir = cfg["run_dir"]
    parameters = {p.split('=')[0]: p.split('=')[1] for p in run_dir.split('/') if "=" in p}

    data = {}
    for yamlf in (run / 'inference' / csv_folder).rglob('*.csv'):
        data[yamlf.parent.stem] = pd.read_csv(yamlf)
    
    data = pd.concat(data, axis=1)
    data.columns.names = ['type', 'metric']
    data.sort_values(by=['type', 'metric'], axis=1, inplace=True)
    data.drop(columns=['epoch', 'step'], level='metric', inplace=True)
    
    if data.empty:
        print(f'Pred folder is empty: {csv_folder}')
    
    for k, v in parameters.items():
        data[k] = v
    
    return data

def collect_all(root, csv_folder):
    root = Path(root)
    metrics = [collect_one(csvf.parents[1], csvf.name) for csvf in list(root.rglob(csv_folder))]
    metrics = pd.concat(metrics, ignore_index=True)
    return metrics

default_fields_dict = {
    'r1': lambda x: u"{:.1f}".format(x),
    'r5': lambda x: u"{:.1f}".format(x),
    'r10': lambda x: u"{:.1f}".format(x),
    'meanr': lambda x: u"{:.1f}".format(x),
    'medr': lambda x: int(x),
    'spice': lambda x: u"{:.3f}".format(x),
    'spacy': lambda x: u"{:.3f}".format(x),
}
def render_to_latex(metrics, rename_func=default_fields_dict, **latex_kwargs):
    m = metrics.copy()
    # renaming
    for col, lambda_fn in rename_func.items():
        m[col] = m[col].apply(lambda_fn)
    # m = m.applymap(lambda x: u"{:.2f}".format(x))
    ltex = m.style.to_latex(
        **latex_kwargs
    )
    return ltex

In [3]:
# Compute metrics for each detected run

def summarize_metrics(
        metrics, 
        dataset=None, 
        model=None,
        translator=None,
        tok_position=None, 
        training_setup=None, 
        loss=None, 
        learning_rate=None, 
        finetuning=None, 
        drop_i2t=True):
    
    if dataset is not None:
        metrics = metrics[(metrics['data'] == dataset)]
        metrics.drop(columns="data", inplace=True)

    # TODO: as of now, there is only one split seed.
    # In the future, we would have to average among different splits
    # metrics.drop(columns="split_seed", inplace=True)

    id_vars = ['data', 'model', 'translator', 'tok_position', 'training-setup', 'loss', 'lr', 'finetuning']

    if translator is not None:
        metrics = metrics[metrics['translator'].isin(learning_rate)]
        if len(translator) == 1:
            metrics.drop(columns="translator", inplace=True)
            id_vars.remove('translator')
    if model is not None:
        metrics = metrics[metrics['model'].isin(learning_rate)]
        if len(model) == 1:
            metrics.drop(columns="model", inplace=True)
            id_vars.remove('model')
    if learning_rate is not None:
        metrics = metrics[metrics['lr'].isin(learning_rate)]
        if len(learning_rate) == 1:
            metrics.drop(columns="lr", inplace=True)
            id_vars.remove('lr')
    if finetuning is not None:
        metrics = metrics[metrics['finetuning'].isin(finetuning)]
        if len(finetuning) == 1:
            metrics.drop(columns="finetuning", inplace=True)
            id_vars.remove('finetuning')
    if tok_position is not None:
        metrics = metrics[metrics['tok_position'].isin(tok_position)]
        if len(tok_position) == 1:
            metrics.drop(columns="tok_position", inplace=True)
            id_vars.remove('tok_position')
    if training_setup is not None:
        metrics = metrics[metrics['training-setup'].isin(training_setup)]
        if len(training_setup) == 1:
            metrics.drop(columns="training-setup", inplace=True)
            id_vars.remove('training-setup')
    if loss is not None:
        metrics = metrics[metrics['loss'].isin(loss)]
        if len(loss) == 1:
            metrics.drop(columns="loss", inplace=True)
            id_vars.remove('loss')

    if drop_i2t:
        # remove columns containing i2t in the name of the second level of the multiindex
        metrics = metrics.loc[:, ~metrics.columns.get_level_values(1).str.contains('i2t')]

    metrics.set_index(id_vars, inplace=True)
    # split into different dataframes, one for each column (first level of the multiindex)
    column_types = list(metrics.columns.get_level_values(0).unique())
    metrics = {c: metrics.loc[:, c].copy() for c in column_types}
    {k: v.columns.set_names(k, inplace=True) for k, v in metrics.items()}
    return metrics

In [4]:
# rename content of the table
def rename_fn(v):
    mapping = {'ContrastiveFixed': 'Triplet',
               'InfoNCELoss': 'InfoNCE'}
    if v in mapping:
        return mapping[v]
    return v

# Results (best contrastive sum checkpoint)

In [5]:
# collect all data
ROOT = "runs"

metrics = collect_all(ROOT, 'best-contrastive-sum')
metrics = summarize_metrics(
    metrics,
    # tok_position=["tok_beginning"],
    )

for k in metrics:
    display(metrics[k])

entities_retrieval                                                                                                                                        entities_kmin_sum  \
data                     model  translator  tok_position                training-setup                                loss     lr    finetuning                               
coco_faceswap_5_entities idclip mlp-1-layer tok_in_place_multi_prompts  with_entities                                 info-nce 5e-05 disabled                      0.639023   
                                                                                                                                     shallow-tpt-5                 0.717757   
                                                                                                                                     shallow-vpt-5-tpt-5           0.760219   
                                                                                                                                     shallow-vpt-5                 0.773800   
                                                                        with_entities_and_only_entities               info-nce 5e-05 disabled                      0.507575   
                                                                                                                                     shallow-tpt-5                 0.493698   
                                                                                                                                     shallow-vpt-5-tpt-5           0.505430   
                                                                                                                                     shallow-vpt-5                 0.586423   
                                            tok_beginning_multi_prompts with_entities_and_only_entities_weighted_0.10 info-nce 5e-05 shallow-vpt-5                 0.405201   
                                                                        with_entities_and_only_entities_weighted_0.02 info-nce 5e-05 shallow-vpt-5                 0.757821   
                                                                        with_entities                                 info-nce 5e-05 disabled                      0.670115   
                                                                                                                                     shallow-tpt-5                 0.709700   
                                                                                                                                     shallow-vpt-5-tpt-5           0.800140   
                                                                                                                                     shallow-vpt-5                 0.779060   
                                                                        with_entities_and_only_entities               info-nce 5e-05 disabled                      0.543555   
                                                                                                                                     shallow-tpt-5                 0.527191   
                                                                                                                                     shallow-vpt-5-tpt-5           0.481218   
                                                                                                                                     shallow-vpt-5                 0.603405   
                                            tok_beginning_fixed_prompt  with_entities                                 info-nce 5e-05 disabled                      0.716909   
                                                                                                                                     disabled                      0.716909   
                                                                        with_entities_and_only_entities               info-nce 5e-05 disabled         

entities_retrieval_prompt_ensemble                                                                                                                        entities_kmin_sum  \
data                     model  translator  tok_position                training-setup                                loss     lr    finetuning                               
coco_faceswap_5_entities idclip mlp-1-layer tok_in_place_multi_prompts  with_entities                                 info-nce 5e-05 disabled                      0.654243   
                                                                                                                                     shallow-tpt-5                 0.714459   
                                                                                                                                     shallow-vpt-5-tpt-5           0.782036   
                                                                                                                                     shallow-vpt-5                 0.773075   
                                                                        with_entities_and_only_entities               info-nce 5e-05 disabled                      0.553252   
                                                                                                                                     shallow-tpt-5                 0.477610   
                                                                                                                                     shallow-vpt-5-tpt-5           0.505803   
                                                                                                                                     shallow-vpt-5                 0.575223   
                                            tok_beginning_multi_prompts with_entities_and_only_entities_weighted_0.10 info-nce 5e-05 shallow-vpt-5                 0.406880   
                                                                        with_entities_and_only_entities_weighted_0.02 info-nce 5e-05 shallow-vpt-5                 0.756139   
                                                                        with_entities                                 info-nce 5e-05 disabled                      0.671665   
                                                                                                                                     shallow-tpt-5                 0.692205   
                                                                                                                                     shallow-vpt-5-tpt-5           0.787078   
                                                                                                                                     shallow-vpt-5                 0.799062   
                                                                        with_entities_and_only_entities               info-nce 5e-05 disabled                      0.549205   
                                                                                                                                     shallow-tpt-5                 0.547905   
                                                                                                                                     shallow-vpt-5-tpt-5           0.518865   
                                                                                                                                     shallow-vpt-5                 0.569473   
                                            tok_beginning_fixed_prompt  with_entities                                 info-nce 5e-05 disabled                      0.652072   
                                                                                                                                     disabled                      0.652072   
                                                                        with_entities_and_only_entities               info-nce 5e-05 disabled         

general_retrieval_tok_beginning                                                                                                                           contrastive_sum  \
data                     model  translator  tok_position                training-setup                                loss     lr    finetuning                             
coco_faceswap_5_entities idclip mlp-1-layer tok_in_place_multi_prompts  with_entities                                 info-nce 5e-05 disabled                    5.697727   
                                                                                                                                     shallow-tpt-5               5.386137   
                                                                                                                                     shallow-vpt-5-tpt-5         5.531250   
                                                                                                                                     shallow-vpt-5               5.629659   
                                                                        with_entities_and_only_entities               info-nce 5e-05 disabled                    5.715682   
                                                                                                                                     shallow-tpt-5               5.442614   
                                                                                                                                     shallow-vpt-5-tpt-5         5.597159   
                                                                                                                                     shallow-vpt-5               5.759773   
                                            tok_beginning_multi_prompts with_entities_and_only_entities_weighted_0.10 info-nce 5e-05 shallow-vpt-5               5.699773   
                                                                        with_entities_and_only_entities_weighted_0.02 info-nce 5e-05 shallow-vpt-5               5.685682   
                                                                        with_entities                                 info-nce 5e-05 disabled                    5.655114   
                                                                                                                                     shallow-tpt-5               5.432045   
                                                                                                                                     shallow-vpt-5-tpt-5         5.557159   
                                                                                                                                     shallow-vpt-5               5.669318   
                                                                        with_entities_and_only_entities               info-nce 5e-05 disabled                    5.642273   
                                                                                                                                     shallow-tpt-5               5.406363   
                                                                                                                                     shallow-vpt-5-tpt-5         5.653296   
                                                                                                                                     shallow-vpt-5               5.725227   
                                            tok_beginning_fixed_prompt  with_entities                                 info-nce 5e-05 disabled                    5.550341   
                                                                                                                                     disabled                    5.550341   
                                                                        with_entities_and_only_entities               info-nce 5e-05 disabled                    5.689773   
                     

general_retrieval_tok_beginning_prompt_ensemble                                                                                                           contrastive_sum  \
data                     model  translator  tok_position                training-setup                                loss     lr    finetuning                             
coco_faceswap_5_entities idclip mlp-1-layer tok_in_place_multi_prompts  with_entities                                 info-nce 5e-05 disabled                    5.686136   
                                                                                                                                     shallow-tpt-5               5.385568   
                                                                                                                                     shallow-vpt-5-tpt-5         5.544773   
                                                                                                                                     shallow-vpt-5               5.614091   
                                                                        with_entities_and_only_entities               info-nce 5e-05 disabled                    5.702841   
                                                                                                                                     shallow-tpt-5               5.460682   
                                                                                                                                     shallow-vpt-5-tpt-5         5.600227   
                                                                                                                                     shallow-vpt-5               5.765341   
                                            tok_beginning_multi_prompts with_entities_and_only_entities_weighted_0.10 info-nce 5e-05 shallow-vpt-5               5.711023   
                                                                        with_entities_and_only_entities_weighted_0.02 info-nce 5e-05 shallow-vpt-5               5.697045   
                                                                        with_entities                                 info-nce 5e-05 disabled                    5.656704   
                                                                                                                                     shallow-tpt-5               5.447386   
                                                                                                                                     shallow-vpt-5-tpt-5         5.543977   
                                                                                                                                     shallow-vpt-5               5.682727   
                                                                        with_entities_and_only_entities               info-nce 5e-05 disabled                    5.659545   
                                                                                                                                     shallow-tpt-5               5.429773   
                                                                                                                                     shallow-vpt-5-tpt-5         5.668182   
                                                                                                                                     shallow-vpt-5               5.754091   
                                            tok_beginning_fixed_prompt  with_entities                                 info-nce 5e-05 disabled                    5.674204   
                                                                                                                                     disabled                    5.674204   
                                                                        with_entities_and_only_entities               info-nce 5e-05 disabled                    5.713295   
                     

general_retrieval_tok_in_place                                                                                                                            contrastive_sum  \
data                     model  translator  tok_position                training-setup                                loss     lr    finetuning                             
coco_faceswap_5_entities idclip mlp-1-layer tok_in_place_multi_prompts  with_entities                                 info-nce 5e-05 disabled                    5.659204   
                                                                                                                                     shallow-tpt-5               5.329545   
                                                                                                                                     shallow-vpt-5-tpt-5         5.478068   
                                                                                                                                     shallow-vpt-5               5.590909   
                                                                        with_entities_and_only_entities               info-nce 5e-05 disabled                    5.643295   
                                                                                                                                     shallow-tpt-5               5.357955   
                                                                                                                                     shallow-vpt-5-tpt-5         5.536023   
                                                                                                                                     shallow-vpt-5               5.682500   
                                            tok_beginning_multi_prompts with_entities_and_only_entities_weighted_0.10 info-nce 5e-05 shallow-vpt-5                    NaN   
                                                                        with_entities_and_only_entities_weighted_0.02 info-nce 5e-05 shallow-vpt-5                    NaN   
                                                                        with_entities                                 info-nce 5e-05 disabled                    5.575455   
                                                                                                                                     shallow-tpt-5               5.329545   
                                                                                                                                     shallow-vpt-5-tpt-5         5.416477   
                                                                                                                                     shallow-vpt-5               5.664659   
                                                                        with_entities_and_only_entities               info-nce 5e-05 disabled                    5.595682   
                                                                                                                                     shallow-tpt-5               5.325227   
                                                                                                                                     shallow-vpt-5-tpt-5         5.536023   
                                                                                                                                     shallow-vpt-5               5.630909   
                                            tok_beginning_fixed_prompt  with_entities                                 info-nce 5e-05 disabled                    5.661136   
                                                                                                                                     disabled                    5.661136   
                                                                        with_entities_and_only_entities               info-nce 5e-05 disabled                    5.649432   
                     

general_retrieval_tok_in_place_prompt_ensemble                                                                                                            contrastive_sum  \
data                     model  translator  tok_position                training-setup                                loss     lr    finetuning                             
coco_faceswap_5_entities idclip mlp-1-layer tok_in_place_multi_prompts  with_entities                                 info-nce 5e-05 disabled                    5.678068   
                                                                                                                                     shallow-tpt-5               5.365568   
                                                                                                                                     shallow-vpt-5-tpt-5         5.527045   
                                                                                                                                     shallow-vpt-5               5.606023   
                                                                        with_entities_and_only_entities               info-nce 5e-05 disabled                    5.680568   
                                                                                                                                     shallow-tpt-5               5.425682   
                                                                                                                                     shallow-vpt-5-tpt-5         5.575114   
                                                                                                                                     shallow-vpt-5               5.727046   
                                            tok_beginning_multi_prompts with_entities_and_only_entities_weighted_0.10 info-nce 5e-05 shallow-vpt-5                    NaN   
                                                                        with_entities_and_only_entities_weighted_0.02 info-nce 5e-05 shallow-vpt-5                    NaN   
                                                                        with_entities                                 info-nce 5e-05 disabled                    5.634091   
                                                                                                                                     shallow-tpt-5               5.411250   
                                                                                                                                     shallow-vpt-5-tpt-5         5.497500   
                                                                                                                                     shallow-vpt-5               5.690909   
                                                                        with_entities_and_only_entities               info-nce 5e-05 disabled                    5.636818   
                                                                                                                                     shallow-tpt-5               5.402727   
                                                                                                                                     shallow-vpt-5-tpt-5         5.631591   
                                                                                                                                     shallow-vpt-5               5.698637   
                                            tok_beginning_fixed_prompt  with_entities                                 info-nce 5e-05 disabled                    5.700796   
                                                                                                                                     disabled                    5.700796   
                                                                        with_entities_and_only_entities               info-nce 5e-05 disabled                    5.697727   
                     

# Results (best entities sum checkpoint)

In [6]:
# collect all data on kit
ROOT = "runs"

metrics = collect_all(ROOT, 'best-entities-sum')
metrics = summarize_metrics(
    metrics,
    # tok_position=["tok_beginning"],
    )

for k in metrics:
    display(metrics[k])

entities_retrieval                                                                                                                                        entities_kmin_sum  \
data                     model  translator  tok_position                training-setup                                loss     lr    finetuning                               
coco_faceswap_5_entities idclip mlp-1-layer tok_in_place_multi_prompts  with_entities                                 info-nce 5e-05 disabled                      0.649747   
                                                                                                                                     shallow-tpt-5                 0.707765   
                                                                                                                                     shallow-vpt-5-tpt-5           0.794221   
                                                                                                                                     shallow-vpt-5                 0.777421   
                                                                        with_entities_and_only_entities               info-nce 5e-05 disabled                      0.553139   
                                                                                                                                     shallow-tpt-5                 0.530989   
                                                                                                                                     shallow-vpt-5-tpt-5           0.540746   
                                                                                                                                     shallow-vpt-5                 0.597779   
                                            tok_beginning_multi_prompts with_entities_and_only_entities_weighted_0.10 info-nce 5e-05 shallow-vpt-5                 0.650057   
                                                                        with_entities_and_only_entities_weighted_0.02 info-nce 5e-05 shallow-vpt-5                 0.747609   
                                                                        with_entities                                 info-nce 5e-05 disabled                      0.702920   
                                                                                                                                     shallow-tpt-5                 0.725525   
                                                                                                                                     shallow-vpt-5-tpt-5           0.813464   
                                                                                                                                     shallow-vpt-5                 0.758440   
                                                                        with_entities_and_only_entities               info-nce 5e-05 disabled                      0.568712   
                                                                                                                                     shallow-tpt-5                 0.540376   
                                                                                                                                     shallow-vpt-5-tpt-5           0.522383   
                                                                                                                                     shallow-vpt-5                 0.597539   
                                            tok_beginning_fixed_prompt  with_entities                                 info-nce 5e-05 disabled                      0.655503   
                                                                                                                                     disabled                      0.655503   
                                                                                                                                     disabled         

entities_retrieval_prompt_ensemble                                                                                                                        entities_kmin_sum  \
data                     model  translator  tok_position                training-setup                                loss     lr    finetuning                               
coco_faceswap_5_entities idclip mlp-1-layer tok_in_place_multi_prompts  with_entities                                 info-nce 5e-05 disabled                      0.643416   
                                                                                                                                     shallow-tpt-5                 0.694876   
                                                                                                                                     shallow-vpt-5-tpt-5           0.795664   
                                                                                                                                     shallow-vpt-5                 0.792312   
                                                                        with_entities_and_only_entities               info-nce 5e-05 disabled                      0.593250   
                                                                                                                                     shallow-tpt-5                 0.506644   
                                                                                                                                     shallow-vpt-5-tpt-5           0.551882   
                                                                                                                                     shallow-vpt-5                 0.598650   
                                            tok_beginning_multi_prompts with_entities_and_only_entities_weighted_0.10 info-nce 5e-05 shallow-vpt-5                 0.641743   
                                                                        with_entities_and_only_entities_weighted_0.02 info-nce 5e-05 shallow-vpt-5                 0.756112   
                                                                        with_entities                                 info-nce 5e-05 disabled                      0.707442   
                                                                                                                                     shallow-tpt-5                 0.701197   
                                                                                                                                     shallow-vpt-5-tpt-5           0.809251   
                                                                                                                                     shallow-vpt-5                 0.769776   
                                                                        with_entities_and_only_entities               info-nce 5e-05 disabled                      0.573108   
                                                                                                                                     shallow-tpt-5                 0.530420   
                                                                                                                                     shallow-vpt-5-tpt-5           0.535698   
                                                                                                                                     shallow-vpt-5                 0.593665   
                                            tok_beginning_fixed_prompt  with_entities                                 info-nce 5e-05 disabled                      0.621858   
                                                                                                                                     disabled                      0.621858   
                                                                                                                                     disabled         

general_retrieval_tok_beginning                                                                                                                           contrastive_sum  \
data                     model  translator  tok_position                training-setup                                loss     lr    finetuning                             
coco_faceswap_5_entities idclip mlp-1-layer tok_in_place_multi_prompts  with_entities                                 info-nce 5e-05 disabled                    5.477159   
                                                                                                                                     shallow-tpt-5               5.334773   
                                                                                                                                     shallow-vpt-5-tpt-5         5.480568   
                                                                                                                                     shallow-vpt-5               5.639545   
                                                                        with_entities_and_only_entities               info-nce 5e-05 disabled                    5.712046   
                                                                                                                                     shallow-tpt-5               5.500454   
                                                                                                                                     shallow-vpt-5-tpt-5         5.450455   
                                                                                                                                     shallow-vpt-5               5.722955   
                                            tok_beginning_multi_prompts with_entities_and_only_entities_weighted_0.10 info-nce 5e-05 shallow-vpt-5               5.753523   
                                                                        with_entities_and_only_entities_weighted_0.02 info-nce 5e-05 shallow-vpt-5               5.690682   
                                                                        with_entities                                 info-nce 5e-05 disabled                    5.542614   
                                                                                                                                     shallow-tpt-5               5.437500   
                                                                                                                                     shallow-vpt-5-tpt-5         5.540795   
                                                                                                                                     shallow-vpt-5               5.644659   
                                                                        with_entities_and_only_entities               info-nce 5e-05 disabled                    5.576818   
                                                                                                                                     shallow-tpt-5               5.480568   
                                                                                                                                     shallow-vpt-5-tpt-5         5.607159   
                                                                                                                                     shallow-vpt-5               5.686477   
                                            tok_beginning_fixed_prompt  with_entities                                 info-nce 5e-05 disabled                    5.703523   
                                                                                                                                     disabled                    5.703523   
                                                                                                                                     disabled                         NaN   
                     

general_retrieval_tok_beginning_prompt_ensemble                                                                                                           contrastive_sum  \
data                     model  translator  tok_position                training-setup                                loss     lr    finetuning                             
coco_faceswap_5_entities idclip mlp-1-layer tok_in_place_multi_prompts  with_entities                                 info-nce 5e-05 disabled                    5.490227   
                                                                                                                                     shallow-tpt-5               5.333750   
                                                                                                                                     shallow-vpt-5-tpt-5         5.514886   
                                                                                                                                     shallow-vpt-5               5.598750   
                                                                        with_entities_and_only_entities               info-nce 5e-05 disabled                    5.715796   
                                                                                                                                     shallow-tpt-5               5.517273   
                                                                                                                                     shallow-vpt-5-tpt-5         5.440909   
                                                                                                                                     shallow-vpt-5               5.710341   
                                            tok_beginning_multi_prompts with_entities_and_only_entities_weighted_0.10 info-nce 5e-05 shallow-vpt-5               5.750000   
                                                                        with_entities_and_only_entities_weighted_0.02 info-nce 5e-05 shallow-vpt-5               5.701477   
                                                                        with_entities                                 info-nce 5e-05 disabled                    5.526477   
                                                                                                                                     shallow-tpt-5               5.442727   
                                                                                                                                     shallow-vpt-5-tpt-5         5.540341   
                                                                                                                                     shallow-vpt-5               5.652386   
                                                                        with_entities_and_only_entities               info-nce 5e-05 disabled                    5.581591   
                                                                                                                                     shallow-tpt-5               5.507614   
                                                                                                                                     shallow-vpt-5-tpt-5         5.614204   
                                                                                                                                     shallow-vpt-5               5.712955   
                                            tok_beginning_fixed_prompt  with_entities                                 info-nce 5e-05 disabled                    5.741477   
                                                                                                                                     disabled                    5.741477   
                                                                                                                                     disabled                         NaN   
                     

general_retrieval_tok_in_place                                                                                                                            contrastive_sum  \
data                     model  translator  tok_position                training-setup                                loss     lr    finetuning                             
coco_faceswap_5_entities idclip mlp-1-layer tok_in_place_multi_prompts  with_entities                                 info-nce 5e-05 disabled                    5.439318   
                                                                                                                                     shallow-tpt-5               5.288977   
                                                                                                                                     shallow-vpt-5-tpt-5         5.453068   
                                                                                                                                     shallow-vpt-5               5.594432   
                                                                        with_entities_and_only_entities               info-nce 5e-05 disabled                    5.645682   
                                                                                                                                     shallow-tpt-5               5.417159   
                                                                                                                                     shallow-vpt-5-tpt-5         5.377841   
                                                                                                                                     shallow-vpt-5               5.631818   
                                            tok_beginning_multi_prompts with_entities_and_only_entities_weighted_0.10 info-nce 5e-05 shallow-vpt-5                    NaN   
                                                                        with_entities_and_only_entities_weighted_0.02 info-nce 5e-05 shallow-vpt-5                    NaN   
                                                                        with_entities                                 info-nce 5e-05 disabled                    5.486477   
                                                                                                                                     shallow-tpt-5               5.322045   
                                                                                                                                     shallow-vpt-5-tpt-5         5.413636   
                                                                                                                                     shallow-vpt-5               5.630000   
                                                                        with_entities_and_only_entities               info-nce 5e-05 disabled                    5.565454   
                                                                                                                                     shallow-tpt-5               5.346478   
                                                                                                                                     shallow-vpt-5-tpt-5         5.516364   
                                                                                                                                     shallow-vpt-5               5.619432   
                                            tok_beginning_fixed_prompt  with_entities                                 info-nce 5e-05 disabled                    5.690682   
                                                                                                                                     disabled                    5.690682   
                                                                                                                                     disabled                    5.690682   
                     

general_retrieval_tok_in_place_prompt_ensemble                                                                                                            contrastive_sum  \
data                     model  translator  tok_position                training-setup                                loss     lr    finetuning                             
coco_faceswap_5_entities idclip mlp-1-layer tok_in_place_multi_prompts  with_entities                                 info-nce 5e-05 disabled                    5.477386   
                                                                                                                                     shallow-tpt-5               5.317955   
                                                                                                                                     shallow-vpt-5-tpt-5         5.497727   
                                                                                                                                     shallow-vpt-5               5.600227   
                                                                        with_entities_and_only_entities               info-nce 5e-05 disabled                    5.692955   
                                                                                                                                     shallow-tpt-5               5.475682   
                                                                                                                                     shallow-vpt-5-tpt-5         5.418636   
                                                                                                                                     shallow-vpt-5               5.677500   
                                            tok_beginning_multi_prompts with_entities_and_only_entities_weighted_0.10 info-nce 5e-05 shallow-vpt-5                    NaN   
                                                                        with_entities_and_only_entities_weighted_0.02 info-nce 5e-05 shallow-vpt-5                    NaN   
                                                                        with_entities                                 info-nce 5e-05 disabled                    5.512159   
                                                                                                                                     shallow-tpt-5               5.403636   
                                                                                                                                     shallow-vpt-5-tpt-5         5.490455   
                                                                                                                                     shallow-vpt-5               5.646477   
                                                                        with_entities_and_only_entities               info-nce 5e-05 disabled                    5.579091   
                                                                                                                                     shallow-tpt-5               5.443523   
                                                                                                                                     shallow-vpt-5-tpt-5         5.587727   
                                                                                                                                     shallow-vpt-5               5.681477   
                                            tok_beginning_fixed_prompt  with_entities                                 info-nce 5e-05 disabled                    5.761818   
                                                                                                                                     disabled                    5.761818   
                                                                                                                                     disabled                         NaN   
                     